In [9]:
import networkx as nx
import pandas as pd
import re

coronavirus = pd.read_csv('#coronavirus_tweets.csv').drop('Unnamed: 0', axis=1)
wuhan = pd.read_csv('wuhan_tweets.csv').drop('Unnamed: 0', axis=1)

tweets = coronavirus.append(wuhan).drop('date', axis=1).drop_duplicates()
#tweets.head(20)

In [10]:
edges = tweets[['username','to']]
edges = edges.ffill(axis=1)
edges['text'] = tweets['text']

# edges['username'] = edges['username'].map(lambda x : x if x not in change_name.items else change_name[x])
# edges['to'] = edges['to'].map(lambda x : x if x not in change_name.items else change_name[x])


# change_name = {'MackayIM':'scientist',
#                'hayesluk': 'scientist',
#                'Laurie_Garret': 'scientist',
#                'microBIOblog': 'scientist',
#                'XavierAbadMdG': 'scientist',
#                'V2019N': 'news',
#                'WHO': 'news',
#                'CDCgov': 'news'}
                
#scientists:  MackayIM, hayesluk, Laurie_Garret, microBIOblog, 'XavierAbadMdG'
#news: V2019N, WHO, CDCgov

In [11]:
G = nx.convert_matrix.from_pandas_edgelist(edges, 'username','to', create_using=nx.DiGraph) 

In [12]:
betweenness = nx.betweenness_centrality(G)
degree = nx.degree_centrality(G)
closeness = nx.closeness_centrality(G)

In [13]:
betweenness_dict = {k: v*100000 for k, v in sorted(betweenness.items(), key=lambda item: item[1], reverse=True)}

In [14]:
{k: v for k, v in sorted(betweenness_dict.items(), key=lambda item: item[1], reverse=True)[:25]}

{'MackayIM': 0.22976419193279973,
 'hayesluk': 0.15257778370537484,
 'HelenBranswell': 0.13372993983588735,
 'arambaut': 0.11847216146534988,
 'Laurie_Garrett': 0.11577961234113736,
 'CentroGripe': 0.10770196496849987,
 'V2019N': 0.07898144097689991,
 'microBIOblog': 0.07359634272847491,
 'DerekGatherer': 0.061928629856887435,
 'greg_folkers': 0.056543531608462434,
 'Crof': 0.037695687738974956,
 'descoladoen': 0.03679817136423746,
 'davidpaulk': 0.030515556741074965,
 'JATetro': 0.021540392993699975,
 'statesdj': 0.01615529474527498,
 'ai6yrham': 0.015257778370537482,
 'NoticiasONU': 0.012565229246324985,
 'doctormacias': 0.010770196496849987,
 'DrTedros': 0.00897516374737499,
 'lookner': 0.00897516374737499,
 'RobRoy_CN': 0.00807764737263749,
 'IndoPac_Info': 0.007180130997899992,
 'iarayah': 0.007180130997899992,
 'BNODesk': 0.006282614623162493,
 'drrandywax': 0.006282614623162493}

In [ ]:
#scientists:  MackayIM, hayesluk, Laurie_Garret, microBIOblog, 'XavierAbadMdG'
#news: V2019N, WHO, CDCgov

In [17]:
betweenness_df = pd.DataFrame(betweenness_dict.items())
betweenness_df.columns = ["Name","Betweenness"]
betweenness_df.head()

,Name,Betweenness
0,MackayIM,0.229764
1,hayesluk,0.152578
2,HelenBranswell,0.133730
3,arambaut,0.118472
4,Laurie_Garrett,0.115780


In [7]:
classes = pd.read_csv('AllPeriod.csv')
classes.head()

,Name,Class
0,MackayIM,S
1,hayesluk,S
2,HelenBranswell,N
3,arambaut,S
4,Laurie_Garrett,S


In [19]:
merged_bet = betweenness_df.merge(classes, left_on = "Name", right_on = "Name")
merged_bet = merged_bet.drop_duplicates()

,Name,Betweenness,Class
0,MackayIM,0.229764,S
2,hayesluk,0.152578,S
4,HelenBranswell,0.133730,N
6,arambaut,0.118472,S
7,Laurie_Garrett,0.115780,S


In [21]:
merged_bet.groupby("Class")["Betweenness"].sum().sort_values(ascending=False)

Class
S    1.339992
N    0.670445
O    0.096034
Name: Betweenness, dtype: float64

In [23]:
degree_dict = {k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}

In [24]:
{k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)[:25]}

{'WHO': 0.00975748389541493,
 'ZulmaCucunuba': 0.0076733611216369835,
 'CDCgov': 0.0038840469874952635,
 'howroute': 0.0033156498673740055,
 'CBCAlerts': 0.0028419856006062904,
 'davidpaulk': 0.0024630541871921183,
 'NPR': 0.002178855627131489,
 'BNONews': 0.001989389920424403,
 'CNN': 0.001989389920424403,
 'AFP': 0.0018946570670708603,
 'V2019N': 0.0017999242137173171,
 'HelenBranswell': 0.0017999242137173171,
 'WHOWPRO': 0.001610458507010231,
 'jenniferatntd': 0.001610458507010231,
 'hayesluk': 0.0015157256536566881,
 'MackayIM': 0.0015157256536566881,
 'cnnbrk': 0.001326259946949602,
 'XavierAbadMdG': 0.0012315270935960591,
 'lookner': 0.0012315270935960591,
 'RFA_Chinese': 0.0012315270935960591,
 'Conflits_FR': 0.001136794240242516,
 'nytimes': 0.001136794240242516,
 'WBYeats1865': 0.001136794240242516,
 'BrownCheong': 0.001136794240242516,
 'Laurie_Garrett': 0.001042061386888973}

In [25]:
pd.DataFrame(degree_dict.items())

,0,1
0,WHO,0.009757
1,ZulmaCucunuba,0.007673
2,CDCgov,0.003884
3,howroute,0.003316
4,CBCAlerts,0.002842
...,...,...
10552,txhurl,0.000095
10553,El__Bohemio,0.000095
10554,ecapobianco,0.000095
10555,Next_Gen_Han,0.000095


In [26]:
betweenness_df = pd.DataFrame(degree_dict.items())
betweenness_df.columns = ["Name","Degree"]

merged_bet = betweenness_df.merge(classes, left_on = "Name", right_on = "Name")
merged_bet = merged_bet.drop_duplicates()

merged_bet.groupby("Class")["Degree"].sum().sort_values(ascending=False)

Class
N    0.047651
S    0.016673
O    0.004452
Name: Degree, dtype: float64

In [28]:
closeness_dict = {k: v for k, v in sorted(closeness.items(), key=lambda item: item[1], reverse=True)}

In [29]:
{k: v for k, v in sorted(closeness.items(), key=lambda item: item[1], reverse=True)[:25]}

{'WHO': 0.009882624825153561,
 'ZulmaCucunuba': 0.007699386630800046,
 'CDCgov': 0.003742914369233862,
 'howroute': 0.003126184160666919,
 'CBCAlerts': 0.002664361500568397,
 'WHOWPRO': 0.002418711149452162,
 'davidpaulk': 0.0023806777929716457,
 'NPR': 0.001989389920424403,
 'BNONews': 0.0018044353019722478,
 'CNN': 0.0017999242137173171,
 'AFP': 0.0017099280030314513,
 'hayesluk': 0.0017051913603637742,
 'catecadell': 0.0016285028600299535,
 'FluTrackers': 0.0015923731440972823,
 'HelenBranswell': 0.0015346722243273969,
 'jenniferatntd': 0.001440936558903891,
 'RFA_Chinese': 0.001367942402425161,
 'rthk_enews': 0.001353326476479186,
 'cnnbrk': 0.0013321807502841986,
 'K_G_Andersen': 0.0012526658294683374,
 'kinlinglo': 0.0011898098737706457,
 'nytimes': 0.001143560872624912,
 'PeterHorby': 0.0011084553524871826,
 'RaynardToddnay1': 0.0010826611811833487,
 'richardneher': 0.0010778493537114226}

In [30]:
pd.DataFrame(closeness_dict.items())

,0,1
0,WHO,0.009883
1,ZulmaCucunuba,0.007699
2,CDCgov,0.003743
3,howroute,0.003126
4,CBCAlerts,0.002664
...,...,...
10552,Next_Gen_Han,0.000000
10553,hanamkim,0.000000
10554,Sally33755,0.000000
10555,sabio28,0.000000


In [31]:
betweenness_df = pd.DataFrame(closeness_dict.items())
betweenness_df.columns = ["Name","Closeness"]

merged_bet = betweenness_df.merge(classes, left_on = "Name", right_on = "Name")
merged_bet = merged_bet.drop_duplicates()

merged_bet.groupby("Class")["Closeness"].sum().sort_values(ascending=False)

Class
N    0.042274
S    0.015065
O    0.002157
Name: Closeness, dtype: float64